In [11]:
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('Financials.csv')

In [13]:
df.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,"$1,618.50",$3.00,$20.00,"$32,370.00",$-,"$32,370.00","$16,185.00","$16,185.00",01/01/2014,1,January,2014
1,Government,Germany,Carretera,None,"$1,321.00",$3.00,$20.00,"$26,420.00",$-,"$26,420.00","$13,210.00","$13,210.00",01/01/2014,1,January,2014
2,Midmarket,France,Carretera,None,"$2,178.00",$3.00,$15.00,"$32,670.00",$-,"$32,670.00","$21,780.00","$10,890.00",01/06/2014,6,June,2014
3,Midmarket,Germany,Carretera,None,$888.00,$3.00,$15.00,"$13,320.00",$-,"$13,320.00","$8,880.00","$4,440.00",01/06/2014,6,June,2014
4,Midmarket,Mexico,Carretera,None,"$2,470.00",$3.00,$15.00,"$37,050.00",$-,"$37,050.00","$24,700.00","$12,350.00",01/06/2014,6,June,2014


In [14]:
#df.columns

df.columns = df.columns.str.strip()

df.columns

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts',
       'Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name',
       'Year'],
      dtype='object')

In [15]:
# Function to convert values in parentheses to negative numbers
def convert_parentheses(val):
    if isinstance(val, str):
        if val.startswith('(') and val.endswith(')'):
            return -float(val[1:-1].replace(',', ''))
        else:
            return float(val)
    else:
        return val

columns_to_clean = ['Units Sold', 'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts', 'Sales', 'COGS', 'Profit']

# Removing '$', '-' and ',' from the columns, handling missing values and negative numbers
for col in columns_to_clean:
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace('$', '').str.replace('-', '').str.replace(',', '').str.strip()
        df[col] = df[col].replace('', np.nan).apply(convert_parentheses).astype(float)

df['Date'] = pd.to_datetime(df['Date'])

In [16]:
df.rename(columns={
    'Sale Price': 'Sale Price ($/unit)',
    'Gross Sales': 'Gross Sales ($)',
    'Discounts': 'Discounts ($)',
    'Sales': 'Net Sales ($)',
    'COGS': 'COGS ($)',
    'Profit': 'Profit ($)'
}, inplace=True)

df.columns

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price ($/unit)', 'Gross Sales ($)',
       'Discounts ($)', 'Net Sales ($)', 'COGS ($)', 'Profit ($)', 'Date',
       'Month Number', 'Month Name', 'Year'],
      dtype='object')

In [17]:
cat_clean = ['Segment', 'Country', 'Product', 'Discount Band', 'Month Name']
for col in cat_clean:
    df[col] = df[col].str.strip().str.title()

In [18]:
"""print(df.isnull().sum())
print(df.describe(include='all'))"""

# Converting object columns to category for categorical variables
category_columns = ['Segment', 'Country', 'Product', 'Discount Band', 'Month Name']
for col in category_columns:
    df[col] = df[col].astype('category')

for col, dtype in df.dtypes.items():
    print(f"{col}: {dtype}")

Segment: category
Country: category
Product: category
Discount Band: category
Units Sold: float64
Manufacturing Price: float64
Sale Price ($/unit): float64
Gross Sales ($): float64
Discounts ($): float64
Net Sales ($): float64
COGS ($): float64
Profit ($): float64
Date: datetime64[ns]
Month Number: int64
Month Name: category
Year: int64


In [19]:
# Adding some useful time parts
df['Quarter'] = ((df['Month Number'] - 1) // 3) + 1
df['Weekday'] = df['Date'].dt.day_name()

# Calculating derived KPIs
df['Profit Margin (%)'] = (df['Profit ($)'] / df['Net Sales ($)']) * 100
df['Discount Rate (%)'] = (df['Discounts ($)'] / df['Gross Sales ($)']) * 100
df['COGS Ratio (%)'] = (df['COGS ($)'] / df['Net Sales ($)']) * 100

# rounding off the new columns to 2 decimal places for clarity
df[['Profit Margin (%)', 'Discount Rate (%)', 'COGS Ratio (%)']] = df[['Profit Margin (%)', 'Discount Rate (%)', 'COGS Ratio (%)']].round(2)

# Saving the updated dataset
df.to_csv("financials_processed.csv", index=False)

# Previewing the new columns
print(df[['Quarter','Weekday','Profit Margin (%)', 'Discount Rate (%)', 'COGS Ratio (%)']].head())


   Quarter    Weekday  Profit Margin (%)  Discount Rate (%)  COGS Ratio (%)
0        1  Wednesday              50.00                NaN           50.00
1        1  Wednesday              50.00                NaN           50.00
2        2     Monday              33.33                NaN           66.67
3        2     Monday              33.33                NaN           66.67
4        2     Monday              33.33                NaN           66.67


In [20]:
df.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price ($/unit),Gross Sales ($),Discounts ($),Net Sales ($),...,Profit ($),Date,Month Number,Month Name,Year,Quarter,Weekday,Profit Margin (%),Discount Rate (%),COGS Ratio (%)
0,Government,Canada,Carretera,None,1618.5,3.0,20.0,32370.0,NaN,32370.0,...,16185.0,2014-01-01,1,January,2014,1,Wednesday,50.00,NaN,50.00
1,Government,Germany,Carretera,None,1321.0,3.0,20.0,26420.0,NaN,26420.0,...,13210.0,2014-01-01,1,January,2014,1,Wednesday,50.00,NaN,50.00
2,Midmarket,France,Carretera,None,2178.0,3.0,15.0,32670.0,NaN,32670.0,...,10890.0,2014-01-06,6,June,2014,2,Monday,33.33,NaN,66.67
3,Midmarket,Germany,Carretera,None,888.0,3.0,15.0,13320.0,NaN,13320.0,...,4440.0,2014-01-06,6,June,2014,2,Monday,33.33,NaN,66.67
4,Midmarket,Mexico,Carretera,None,2470.0,3.0,15.0,37050.0,NaN,37050.0,...,12350.0,2014-01-06,6,June,2014,2,Monday,33.33,NaN,66.67
